## Probemos un poquito Learning to Rank con la librería LightGBM

Seguimos el ejemplo del código en https://mlexplained.com/2019/05/27/learning-to-rank-explained-with-code/

Para eso hay que descargar los datos con el archivo trans_data.py, ejecutando retrieve_30k.sh

#### Para Linux

Si el sistema que corren es Linux, se puede ejecutar la celda siguiente.

In [1]:
! sh retrieve_30k.sh

--2019-11-30 12:55:59--  https://s3-us-west-2.amazonaws.com/xgboost-examples/MQ2008.rar%0D
Resolviendo s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.229.40
Conectando con s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)[52.218.229.40]:443... conectado.
Petición HTTP enviada, esperando respuesta... 403 Forbidden
2019-11-30 12:56:00 ERROR 403: Forbidden.

retrieve_30k.sh: 3: retrieve_30k.sh: unrar: not found
mv: no se puede efectuar `stat' sobre 'MQ2008/Fold1/*.txt': No existe el archivo o el directorio
Traceback (most recent call last):
  File "trans_data.py", line 18, in <module>
    fi = open(sys.argv[1])
FileNotFoundError: [Errno 2] No such file or directory: 'train.txt'
Traceback (most recent call last):
  File "trans_data.py", line 18, in <module>
    fi = open(sys.argv[1])
FileNotFoundError: [Errno 2] No such file or directory: 'test.txt'
Traceback (most recent call last):
  File "trans_data.py", line 18, in <module>
    fi = open(sys.argv[1])
FileNotFou

#### Para Windows

En el caso de Windows, deben tener instalado [7zip](https://www.7-zip.org/) primero. Luego deberán ejecutar las siguientes celdas.

In [4]:
!pip install patool

In [5]:
import os
import patoolib
import requests

rarfile = requests.get("https://s3-us-west-2.amazonaws.com/xgboost-examples/MQ2008.rar")
with open("./MQ2008.rar", "wb") as fh:
    fh.write(rarfile.content)

patoolib.extract_archive("./MQ2008.rar", outdir="./")
os.system("move /-y MQ2008\Fold1\*.txt .")

patool: Extracting ./MQ2008.rar ...


PatoolError: could not find an executable program to extract format rar; candidates are (rar,unrar,7z),

In [ ]:
!python trans_data.py train.txt mq2008.train mq2008.train.group

In [ ]:
!python trans_data.py test.txt mq2008.test mq2008.test.group

In [ ]:
!python trans_data.py vali.txt mq2008.vali mq2008.vali.group

## Learning to Rank

In [ ]:
# Importemos las librerías más importantes
import lightgbm as lgb
import numpy as np
from sklearn.datasets import load_svmlight_file
from scipy.stats import spearmanr

# Carguemos los archivos que pudimos bajar con el script trans_data.py
x_train, y_train = load_svmlight_file("mq2008.train")
x_valid, y_valid = load_svmlight_file("mq2008.vali")
x_test, y_test = load_svmlight_file("mq2008.test")

In [ ]:
y_train

In [ ]:
q_train = np.loadtxt('mq2008.train.group')
q_valid = np.loadtxt('mq2008.vali.group')
q_test = np.loadtxt('mq2008.test.group')

In [ ]:
x_test

In [ ]:
q_test

In [ ]:
y_test[:8]

In [ ]:
# LGBMRanker doc: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRanker.html
gbm = lgb.LGBMRanker()

gbm.fit(
    x_train, y_train, group=q_train, eval_set=[(x_valid, y_valid)],
    eval_group=[q_valid], eval_at=[1, 3], early_stopping_rounds=20, verbose=True,
    callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)]
)

In [ ]:
# Tiremos el predictor sobre los datos de test
preds_test = gbm.predict(x_test)
preds_test

In [ ]:
# Usemos la métrica de Spearman para correlación de Rankings
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html
spearmanr(y_test, preds_test)

## Agrupemos todo el dataset y reentrenemos!

In [ ]:
q_train = [x_train.shape[0]]
q_valid = [x_valid.shape[0]]
q_test = [x_test.shape[0]]

gbm = lgb.LGBMRanker()
gbm.fit(
    x_train, y_train, group=q_train, eval_set=[(x_valid, y_valid)],
    eval_group=[q_valid], eval_at=[1, 3], early_stopping_rounds=20, verbose=True,
    callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)]
)

In [ ]:
preds_test = gbm.predict(x_test)
preds_test
spearmanr(y_test, preds_test)